In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import pymongo file
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# username and password to access the database
username = "aacuser"
password = "92050"
# pyMongo object for getting data from the database
db = AnimalShelter.AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIXED Place the HTML image tag in the line below into the app.layout code according to your design
#FIXED ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Remy Welham: CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style = {'height': '150px'}),
    html.Hr(),
    html.Div(
        
#FIXED Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id = 'filter-type',
            options = [
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain Rescue', 'value': 'mountain_rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster_rescue'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value = 'reset'
            
        )

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
        # features for your interactive data table to make it user-friendly for your client
        editable = True,
        row_selectable = "single", # allows one row to be selectable
        selected_rows = [],
        filter_action = "native", # allows for a filter
        sort_action = "native", # allows for sorting
        page_action = "native",
        page_current = 0, # sets start page
        page_size = 10 # sets how many rows per page

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIXED Add code to filter interactive data table with MongoDB queries
    query = {}
    if filter_type == 'water_rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["Newfoundland", "Labrador Retriever", "Golden Retriever"]}}
    elif filter_type == 'mountain_rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Great Pyrenees", "Saint Bernard"]}}
    elif filter_type == 'disaster_rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever",  "Bloodhound", "German Shepherd", 
                                                     "Border Collie", "Belgian Malinois"]}}
    elif filter_type == 'reset':
        query = {}
        
    
    data = pd.DataFrame.from_records(db.read(query))
    # data.drop(columns=['_id'], inplace = True)
    return data.to_dict('records')
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return html.Div("No data available to display.")
    
    df = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width':'1000px', 'height':'500px'}, 
               center = [30.75, -97.48], zoom = 10, children=[
            #location fallback values
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(
                position=[float(dff.iloc[row, 13]), float(dff.iloc[row, 14])], # lat and long values
                children=[
                dl.Tooltip(dff.iloc[row, 4]  if pd.notna(dff.iloc[row,4])else"unknown"), # breed and fallback
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9] if pd.notna(dff.iloc[row,9])else"No name") # name and fallback
                ])
            ])
        ])
    ]



app.run_server(debug=True, port = 13789)


Dash app running on http://127.0.0.1:13789/
